In [1]:
from collections import defaultdict
import re
import time
import pandas as pd
import altair as alt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

from preprocessing import *

**Go to the end of the notebook for a summary of the analysis**

In [7]:
#pd.set_option('display.max_columns', 50)
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

Load the data:

In [8]:
df = pd.read_csv('../data/train_data.zip')

In [9]:
df.head()

,external_id,month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,...,avg_wind_9_10,avg_wind_10_11,avg_wind_11_12,avg_wind_12_above,perfect_days,unacast_session_count,hpi,state_and_local_amount_per_capita,state_amount_per_capita,local_amount_per_capita
0,1900203,3,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,78.0,323.61,0.132207,0.018519,0.113688
1,1900203,6,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,111.0,323.61,0.132207,0.018519,0.113688
2,1900203,8,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,110.0,323.61,0.132207,0.018519,0.113688
3,MR00101775,1,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,110.38,0.076247,0.011966,0.064281
4,MR00101775,8,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,110.38,0.076247,0.011966,0.064281


Apply basic preprocessing:

In [10]:
# Impute NaN values and drop unnecessary columns
data = preprocessing_na(df)

In [11]:
# Perform one-hot encoding on remaining categorical features
data = clean_categorical(data)

In [12]:
data.head()

,external_id,month,year,B20004e10,B11016e1,B12001e12,B20004e11,B19125e1,B12001e13,B23008e22,...,monthly_Thursday,HI,LI,MI,HD,LD,MD,A,C,D
0,1900203,3,2019,51111,1868,688,0,78934,1342,0,...,0.0,1,0,0,1,0,0,0,1,0
1,1900203,6,2018,51111,1868,688,0,78934,1342,0,...,0.0,1,0,0,1,0,0,0,1,0
2,1900203,8,2018,51111,1868,688,0,78934,1342,0,...,0.0,1,0,0,1,0,0,0,1,0
3,MR00101775,1,2019,45484,2613,980,30417,45578,1097,66,...,0.0,0,1,0,0,1,0,0,1,0
4,MR00101775,8,2019,45484,2613,980,30417,45578,1097,66,...,0.0,0,1,0,0,1,0,0,1,0


In [14]:
# Check shape of output data
data.shape

(50120, 821)

Create `X` and `y`:

In [23]:
X = data.drop('unacast_session_count', axis=1)
y = data.loc[:, 'unacast_session_count']

In [24]:
# For now, drop `external_id` and `state`
X = X.drop(['external_id', 'state'], axis=1)

In [25]:
# Check if there are missing values in X
X.isna().sum().sort_values(ascending=False)

D                     0
B11001e5              0
B11005e9              0
B13016e5              0
B23025e6              0
                     ..
avg_wind_0_1          0
precip_mm_10_above    0
precip_mm_1_10        0
precip_mm_0_1         0
month                 0
Length: 818, dtype: int64

In [26]:
# Check if there are missing values in y
y.isna().sum()

0

No `NaN` values in `X` and `y` - that's good.

Split the data into training and validation sets:

In [27]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.2,
                                                      random_state=2020)

In [28]:
X_train.shape

(40096, 818)

In [29]:
X_valid.shape

(10024, 818)

Fit a `GradientBoostingRegressor` with default settings:

In [151]:
params = {'verbose': 1,
          'random_state': 2020}

In [152]:
gbr = GradientBoostingRegressor(**params)
gbr.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1      213669.2304            4.86m
         2      205282.8246            4.79m
         3      198390.9645            4.72m
         4      191658.6878            5.01m
         5      186589.0004            4.92m
         6      181186.5017            4.90m
         7      177208.5001            4.84m
         8      174242.0719            5.04m
         9      171638.3291            5.22m
        10      165022.6046            5.32m
        20      129466.3909            4.65m
        30       99860.7850            3.98m
        40       81710.4135            3.35m
        50       70320.0408            2.81m
        60       62436.7513            2.22m
        70       58201.8945            1.67m
        80       53252.5016            1.11m
        90       49244.3230           33.05s
       100       46957.9923            0.00s


In [170]:
# Calculate validation MSE
y_pred = gbr.predict(X_valid)
mean_squared_error(y_valid, y_pred)

370204.9588049915

In [153]:
# Calculate R^2 of the prediction on the validation set
gbr.score(X_valid, y_valid)

0.27425744561888155

Create another `GradientBoostingRegressor` where `n_estimators` is increased to 1000.

In [154]:
params_1000 = {'verbose': 1,
               'n_estimators': 1000,
               'random_state': 2020}

In [155]:
gbr_1000 = GradientBoostingRegressor(**params_1000)

t0 = time.time()
gbr_1000.fit(X_train, y_train)
t1 = time.time()
fit_time = t1 - t0

      Iter       Train Loss   Remaining Time 
         1      213669.2304           48.80m
         2      205282.8246           47.63m
         3      198390.9645           47.96m
         4      191658.6878           49.80m
         5      186589.0004           53.81m
         6      181186.5017           53.67m
         7      177208.5001           53.37m
         8      174242.0719           53.11m
         9      171638.3291           52.80m
        10      165022.6046           52.90m
        20      129466.3909           50.69m
        30       99860.7850           51.99m
        40       81710.4135           51.96m
        50       70320.0408           51.28m
        60       62436.7513           53.76m
        70       58201.8945           53.96m
        80       53252.5016           52.57m
        90       49244.3230           50.99m
       100       46957.9923           49.76m
       200       33108.7092           43.56m
       300       26537.4012           37.29m
       40

In [171]:
# Calculate validation MSE
y_pred_1000 = gbr_1000.predict(X_valid)
mean_squared_error(y_valid, y_pred_1000)

364149.91425749223

In [162]:
# Calculate R^2 of the prediction on the validation set
gbr_1000.score(X_valid, y_valid)

0.2861276364206973

Perform randomized search of optimal hyperparameters:

In [23]:
param_grid = {'min_samples_split': [2, 4, 6],
              'max_depth': [3, 5, 7, 9],
              'max_features': ['auto', 'sqrt']}

In [25]:
gbr_gs = GradientBoostingRegressor(n_estimators=1000, verbose=1, random_state=2020)

rscv = RandomizedSearchCV(gbr_gs, param_grid, n_iter=3, verbose=2, cv=3, n_jobs=1, random_state=2020)

search = rscv.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] min_samples_split=4, max_features=auto, max_depth=3 .............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


      Iter       Train Loss   Remaining Time 
         1      153317.5171           65.14m
         2      142882.3344           66.35m
         3      135685.8230           66.56m
         4      128130.1997           66.66m
         5      122470.8207           66.94m
         6      118451.6037           66.95m
         7      115259.8875           67.29m
         8      112609.5008           67.27m
         9      109552.1000           67.53m
        10      107623.1704           68.16m
        20       89430.5067           67.99m
        30       71334.1225           69.04m
        40       60044.3450           69.81m
        50       52779.6393           69.76m
        60       47767.4358           69.39m
        70       43246.5435           68.77m
        80       40850.4885           68.26m
        90       39042.1967           67.94m
       100       37326.2071           67.25m
       200       24353.1053           53.62m
       300       17797.7181           38.50m
       40

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 37.7min remaining:    0.0s


      Iter       Train Loss   Remaining Time 
         1      241967.4629           30.65m
         2      225379.2903           30.43m
         3      208913.0207           30.43m
         4      201653.9873           30.33m
         5      188521.9523           30.30m
         6      177523.3656           30.27m
         7      166659.6284           30.17m
         8      158616.3034           30.14m
         9      150653.1653           30.08m
        10      143361.2094           30.05m
        20      107967.1941           29.69m
        30       81686.2252           29.74m
        40       65468.1422           29.55m
        50       54354.3595           29.37m
        60       48476.4330           29.05m
        70       45189.1124           28.73m
        80       42161.0303           28.38m
        90       39027.2504           28.08m
       100       36137.7814           27.76m
       200       25283.7676           24.63m
       300       19759.4666           21.70m
       40

       300        1952.7888            1.80m
       400        1389.6867            1.54m
       500        1055.7018            1.29m
       600         842.7764            1.03m
       700         665.2847           46.73s
       800         535.4185           31.21s
       900         448.4783           15.64s
      1000         372.9179            0.00s
[CV]  min_samples_split=2, max_features=sqrt, max_depth=7, total= 2.6min
[CV] min_samples_split=2, max_features=sqrt, max_depth=7 .............
      Iter       Train Loss   Remaining Time 
         1      231119.0006            2.55m
         2      206885.3001            2.52m
         3      186561.3483            2.49m
         4      162252.3419            2.50m
         5      141641.5785            2.48m
         6      126785.6327            2.48m
         7      113329.5210            2.46m
         8      101487.7961            2.46m
         9       89886.4764            2.46m
        10       80677.5757            2.45m


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 365.4min finished


      Iter       Train Loss   Remaining Time 
         1      201432.3232            3.71m
         2      187698.6130            3.69m
         3      172924.2830            3.59m
         4      165930.4164            3.59m
         5      152832.8956            3.57m
         6      137973.6947            3.56m
         7      129007.9530            3.54m
         8      119984.4184            3.54m
         9      110339.4638            3.54m
        10      100848.2244            3.51m
        20       54586.3249            3.48m
        30       35933.6346            3.48m
        40       25765.4375            3.47m
        50       19896.6681            3.44m
        60       15900.4821            3.40m
        70       13601.7552            3.39m
        80       11404.3946            3.36m
        90       10302.6814            3.33m
       100        9233.4724            3.29m
       200        4295.5839            2.94m
       300        2765.3633            2.57m
       40

In [31]:
# Print the most optimal hyperparameter settings
search.best_params_

{'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': 7}

In [32]:
# Print the evaluation of the hyperparameter candidates
result = search.cv_results_
pd.DataFrame(result)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_split,param_max_features,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,2159.919200,73.135515,1.049887,0.263267,4,auto,3,"{'min_samples_split': 4, 'max_features': 'auto...",0.161272,0.364089,0.055305,0.193555,0.128111,3
1,4987.493293,430.139624,1.645919,0.316282,4,auto,7,"{'min_samples_split': 4, 'max_features': 'auto...",0.190472,0.284677,0.402996,0.292715,0.086948,2
2,157.151442,0.092279,1.505860,0.004716,2,sqrt,7,"{'min_samples_split': 2, 'max_features': 'sqrt...",0.128976,0.545482,0.384348,0.352935,0.171483,1


In [35]:
# Call predict on the most optimal hyperparameter on the training set
# Calculate training MSE
mean_squared_error(y_train, search.predict(X_train))

651.2234780126952

In [36]:
# Calculate R^2 of the prediction on the training set
search.score(X_train, y_train)

0.9971445104080854

In [37]:
# Call predict on the most optimal hyperparameter on the validation set
# Calculate validation MSE
mean_squared_error(y_valid, search.predict(X_valid))

394161.808067091

In [39]:
# Calculate R^2 of the prediction on the validation set
search.score(X_valid, y_valid)

0.2272929072871356

Try removing the playgrounds with over 70,000 lifetime sessions.

In [13]:
# Create a list of playgrounds to remove
popular_sites = ['9322b009-a5d3-4c74-b355-5f044b449890', 'MR00096951', '1804229', '44aae0ac-f1b2-433b-961c-546c54867f26',
                 'MR00103365', 'MR00115762', 'MR00101404', 'FM00174462', 'FM00169129']

In [14]:
# Drop these two playgrounds from the input data
new_data = data.copy()
new_data = new_data[~ new_data['external_id'].isin(popular_sites)]
new_data.shape

(49940, 821)

In [15]:
# Create X and Y, split into training and validation sets
new_X = new_data.drop('unacast_session_count', axis=1)
new_y = new_data.loc[:, 'unacast_session_count']

# For now, drop `external_id` and `state`
new_X = new_X.drop(['external_id', 'state'], axis=1)

new_X_train, new_X_valid, new_y_train, new_y_valid = train_test_split(new_X, new_y, 
                                                                      test_size=0.2,
                                                                      random_state=2020)

Fit a `GradientBoostingRegressor` with default settings:

In [49]:
gbr_ps = GradientBoostingRegressor(verbose=1, random_state=2020)
gbr_ps.fit(new_X_train, new_y_train)

      Iter       Train Loss   Remaining Time 
         1       72059.0349            4.58m
         2       67479.8754            4.61m
         3       63647.6264            4.55m
         4       60488.9997            4.56m
         5       57637.7708            4.49m
         6       55323.6846            4.46m
         7       53430.0945            4.70m
         8       51802.9598            4.89m
         9       50416.7531            4.92m
        10       49270.4861            4.89m
        20       42582.4384            4.21m
        30       38146.3113            3.75m
        40       35631.3491            3.15m
        50       33321.8536            2.66m
        60       31242.3235            2.12m
        70       29154.8346            1.63m
        80       28138.6953            1.10m
        90       26996.9965           32.61s
       100       26136.0349            0.00s


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=2020, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=1, warm_start=False)

In [18]:
def report_performance(model, X_train, y_train, X_valid, y_valid):
    """
    Evaluate train and validation performance on a fitted model.
    
    Parameters
    ---------     
    model: sklearn.ensemble._gb.GradientBoostingRegressor
        scikit-learn model
    X_train: pandas.core.frame.DataFrame
        X of training set
    y_train: pandas.core.series.Series
        y of training set
    X_valid: pandas.core.frame.DataFrame        
        X of validation set
    y_valid: pandas.core.series.Series
        y of validation set     
    
    Returns
    -------
    errors: list
        
    """
    errors = [mean_squared_error(y_train, model.predict(X_train)), 
              mean_squared_error(y_valid, model.predict(X_valid))]
    
    print('Training mean squared error:', errors[0])
    print('Validation mean squared error:', errors[1])

In [70]:
report_performance(gbr_ps, new_X_train, new_y_train, new_X_valid, new_y_valid)

Training mean squared error: 26136.034904114495
Validation mean squared error: 22786.46644219538


With the same data, create another `GradientBoostingRegressor` where `n_estimators` is increased to 1000.

In [74]:
gbr_ps_1000 = GradientBoostingRegressor(n_estimators=1000, verbose=1, random_state=2020)
gbr_ps_1000.fit(new_X_train, new_y_train)

      Iter       Train Loss   Remaining Time 
         1       72059.0349           54.59m
         2       67479.8754           58.34m
         3       63647.6264           58.31m
         4       60488.9997           58.19m
         5       57637.7708           58.43m
         6       55323.6846           58.36m
         7       53430.0945           58.10m
         8       51802.9598           58.09m
         9       50416.7531           57.63m
        10       49270.4861           57.29m
        20       42582.4384           56.48m
        30       38146.3113           60.49m
        40       35631.3491           59.07m
        50       33321.8536           57.77m
        60       31242.3235           56.38m
        70       29154.8346           55.39m
        80       28138.6953           56.23m
        90       26996.9965           56.51m
       100       26136.0349           55.57m
       200       19999.9568           45.09m
       300       16288.3391           37.19m
       40

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=2020, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=1, warm_start=False)

In [75]:
report_performance(gbr_ps_1000, new_X_train, new_y_train, new_X_valid, new_y_valid)

Training mean squared error: 7246.417608647951
Validation mean squared error: 17570.52261992865


Try creating a rudimentary `XGBRegressor`:

In [16]:
xgbr = XGBRegressor()
t0 = time.time()
xgbr.fit(new_X_train, new_y_train)
t1 = time.time()
fit_time = t1 - t0

In [19]:
report_performance(xgbr, new_X_train, new_y_train, new_X_valid, new_y_valid)

Training mean squared error: 2865.0722273839897
Validation mean squared error: 15932.369947959685


In [20]:
# Print how long it took to fit the model (in seconds)
print(fit_time)

116.27326512336731


**Summary of the preliminary analysis**

- In this preliminary round of modeling, two different regressors were considered: `GradientBoostingRegressor` and `XGBRegressor`
- When a `GradientBoostingRegressor` with the default settings (i.e. `n_estimators=100`) were fit to the preprocessed data, the validation RMSE was 608. Considering that the RMSE of the initial linear regression model (fit by George) was 289, the results suggests that the initial boosting model performed poorly.
- Next, basic hyperparameter tuning was performed using `RandomizedSearchCV`. For all models, `n_estimator=1000`. 
    - `{'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': 7}` generated the best results.
    - The best model appeared to be overfit. Although gradient boosting is robust to overfitting, `n_estimators` could be lowered. 
    - The validation RMSE of the best model was 628.
- After finding out that there are some playgrounds in the dataset with historic session counts greater than 70,000, another round of modeling was performed with these playgrounds removed from the data.
    - `GradientBoostingRegressor` with default settings:
        - Validation RMSE: 150
    - `GradientBoostingRegressor` with `n_estimator=1000`:
        - Validation RMSE: 132
    - `XGBRegressor` with default settings:
        - Validation RMSE: 150
        - Fitting took less time than scikit-learn's implementation
- This initial analysis revealed that the skewness of the data is contributing to high MSE values. Follow-up is required as to why some playgrounds have incredibly high play session counts.

References:
- https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html
- https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/
- http://www.chengli.io/tutorials/gradient_boosting.pdf